In [ ]:
python src/tools/image_slicer.py --image_path 2025_08_05_PhotoRieboR4_g201b201078_f003_259.JPG --overlap_percentage 30 --destination_folder ./slices/ --slice-size 512

In [8]:
!nvidia-smi

Thu Sep 25 09:47:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090      WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   49C    P8             37W /  600W |    3410MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [101]:
import torch

print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")
    print(f"Количество GPU: {torch.cuda.device_count()}")
    print(f"Текущий GPU device: {torch.cuda.current_device()}")
    device = 'cuda'
else:
    print("CUDA недоступна, используется CPU")
    device = 'cpu'


CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0


In [102]:
from ultralytics import YOLO

model_name = "yolo11x"
model = YOLO(f'{model_name}.pt')
model.to(device)
print(f"Модель загружена на: {device}")

Модель загружена на: cuda


In [103]:
image_path = "2025_08_05_PhotoRieboR4_g201b201078_f003_259.JPG"

In [47]:
slice_path = r"slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_007_007_3227_3227.png"

In [48]:
predictions = model.predict(slice_path, conf=0.1, device=device)
print(f"Количество детекций: {len(predictions[0].boxes.conf)}")
print(f"Используемое устройство: {device}")
print(predictions[0].boxes.xyxy)
list_of_boxes = predictions[0].boxes.xyxy.tolist()


image 1/1 c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\slices\2025_08_05_PhotoRieboR4_g201b201078_f003_259\2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_007_007_3227_3227.png: 640x640 6 birds, 28.6ms
Speed: 3.6ms preprocess, 28.6ms inference, 5.2ms postprocess per image at shape (1, 3, 640, 640)
Количество детекций: 6
Используемое устройство: cuda
tensor([[3.3217e+02, 1.2781e-01, 3.6921e+02, 4.3585e+01],
        [3.6726e+02, 2.9117e+02, 4.0825e+02, 3.1028e+02],
        [3.2111e+02, 4.2453e+02, 3.5456e+02, 4.5206e+02],
        [2.8269e+02, 4.0887e+02, 3.1934e+02, 4.4877e+02],
        [4.0217e+02, 3.7962e+02, 4.2558e+02, 4.0467e+02],
        [4.7605e+02, 4.0503e+02, 5.1181e+02, 4.3833e+02]], device='cuda:0')


In [17]:
predictions[0].save("predicted_image.png")

'predicted_image.png'

In [49]:
images_folder = "slices"

In [104]:
import os
import time

output_folder = f"predicted_images-{model_name}"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"Обработка изображений на устройстве: {device}")
start_time = time.time()
processed_count = 0
total_detections = 0
boxes_list = []
for folder in os.listdir(images_folder):
    folder_path = f"{images_folder}/{folder}"
    if os.path.isdir(folder_path):
        for image_path in os.listdir(folder_path):
            full_image_path = f"{folder_path}/{image_path}"
            predictions = model.predict(full_image_path, conf=0.1, device=device, verbose=False)
            
            if len(predictions[0].boxes.conf) > 0:
                boxes_list.append({'source_image_path': full_image_path, 'coordinates': predictions[0].boxes.xyxy.tolist()})
                predictions[0].save(f"{output_folder}/{image_path}")
                processed_count += 1
                total_detections += len(predictions[0].boxes.conf)
                if processed_count % 10 == 0:  
                    print(f"Обработано: {processed_count} изображений с детекциями")

end_time = time.time()
print(f"\nГотово! Обработано {processed_count} изображений с детекциями")
print(f"Общее количество детекций: {total_detections}")
print(f"Время обработки: {end_time - start_time:.2f} секунд")
print(f"Устройство: {device}")


Обработка изображений на устройстве: cuda
Обработано: 10 изображений с детекциями

Готово! Обработано 11 изображений с детекциями
Общее количество детекций: 42
Время обработки: 4.00 секунд
Устройство: cuda


In [105]:
boxes_list

[{'source_image_path': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_003_015_6915_1383.png',
  'coordinates': [[222.8046875,
    279.7713317871094,
    235.0144500732422,
    296.3580017089844]]},
 {'source_image_path': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_006_007_3227_2766.png',
  'coordinates': [[337.6643371582031,
    458.8148498535156,
    369.6338195800781,
    504.7015686035156],
   [481.3793029785156,
    374.1410827636719,
    511.8021545410156,
    405.4268493652344]]},
 {'source_image_path': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_006_008_3688_2766.png',
  'coordinates': [[110.2598648071289,
    333.608642578125,
    142.728759765625,
    364.2553405761719],
   [20.851703643798828,
    374.2301940917969,
    58.78338623046875,
    404.7321472167969]]},
 {'source_image_path': 'slices/2025_08

In [106]:
for boxes in boxes_list:
    x_min = int(boxes['source_image_path'].split('/')[-1].split('_')[-2].split('.')[0])
    y_min = int(boxes['source_image_path'].split('/')[-1].split('_')[-1].split('.')[0])

    for box in boxes['coordinates']:
        box[0] = box[0] + x_min
        box[1] = box[1] + y_min
        box[2] = box[2] + x_min
        box[3] = box[3] + y_min

boxes_list

[{'source_image_path': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_003_015_6915_1383.png',
  'coordinates': [[7137.8046875,
    1662.7713317871094,
    7150.014450073242,
    1679.3580017089844]]},
 {'source_image_path': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_006_007_3227_2766.png',
  'coordinates': [[3564.664337158203,
    3224.8148498535156,
    3596.633819580078,
    3270.7015686035156],
   [3708.3793029785156,
    3140.141082763672,
    3738.8021545410156,
    3171.4268493652344]]},
 {'source_image_path': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_006_008_3688_2766.png',
  'coordinates': [[3798.259864807129,
    3099.608642578125,
    3830.728759765625,
    3130.255340576172],
   [3708.851703643799,
    3140.230194091797,
    3746.7833862304688,
    3170.732147216797]]},
 {'source_image_path': 'slic

In [131]:
# Импорты уже выполнены в предыдущей ячейке
def calculate_iou(box1, box2):
    # box = [x1, y1, x2, y2]
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    print("intersection_area", intersection_area)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area
    print("union_area", union_area)

    if union_area == 0:
        return 0.0

    return intersection_area / union_area

threshold = 0.7

all_boxes = []
for box_group in boxes_list:
    source_path = box_group['source_image_path']
    for coord in box_group['coordinates']:
        all_boxes.append({
            'coordinates': coord,
            'source': source_path
        })

with_duplicates = len(all_boxes)
print(f"Всего боксов для сравнения: {with_duplicates}")
count = 0
for i in range(len(all_boxes)):
    for j in range(i + 1, len(all_boxes)):
        iou = calculate_iou(all_boxes[i]['coordinates'], all_boxes[j]['coordinates'])
        # print("iou", iou)
        if iou > threshold:
            print(f"Боксы с IoU > {threshold}:")
            print(f"Бокс 1: {all_boxes[i]['coordinates']} (из {all_boxes[i]['source']})")
            print(f"Бокс 2: {all_boxes[j]['coordinates']} (из {all_boxes[j]['source']})")         
            print(f"IoU: {iou:.4f}")
            print("---")
            count += 1

print(f"Количество боксов с IoU > {threshold}: {count}")
without_duplicates = len(all_boxes) - count
print(f"Количество боксов без дубликатов: {without_duplicates}")

Всего боксов для сравнения: 42
intersection_area 1370.7773053476694
union_area 1705.672730034751
Боксы с IoU > 0.7:
Бокс 1: [3564.664337158203, 3224.8148498535156, 3596.633819580078, 3270.7015686035156] (из slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_006_007_3227_2766.png)
Бокс 2: [3559.171600341797, 3227.1278106719255, 3596.2076110839844, 3270.5848503112793] (из slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_007_007_3227_3227.png)
IoU: 0.8037
---
intersection_area 913.5472493395209
union_area 1195.2453882098198
Боксы с IoU > 0.7:
Бокс 1: [3708.3793029785156, 3140.141082763672, 3738.8021545410156, 3171.4268493652344] (из slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_006_007_3227_2766.png)
Бокс 2: [3708.851703643799, 3140.230194091797, 3746.7833862304688, 3170.732147216797] (из slices/2025_08_05_PhotoRieboR4_g201b2010

In [140]:
import numpy as np

sum_of_areas = 0
areas = []
for box in all_boxes:
    box_area = (box['coordinates'][2] - box['coordinates'][0]) * (box['coordinates'][3] - box['coordinates'][1])
    sum_of_areas += box_area
    areas.append(box_area)
print("sum_of_areas", sum_of_areas)
mean_of_areas = sum_of_areas / len(all_boxes)
print("mean_of_areas", mean_of_areas)
iqr = np.percentile(areas, 75) - np.percentile(areas, 25)
print("iqr", iqr)
without_duplicates_and_outliers = without_duplicates
for box in all_boxes:
    box_area = (box['coordinates'][2] - box['coordinates'][0]) * (box['coordinates'][3] - box['coordinates'][1])
    if box_area > mean_of_areas + 3 * iqr:
        print("plus outliersbox_area", box_area)
        print("box", box)
        print("---")
        without_duplicates_and_outliers += 1
    if box_area < mean_of_areas - 3 * iqr:
        print("minus outliers box_area", box_area)
        print("box", box)
        print("---")
        without_duplicates_and_outliers -= 1


sum_of_areas 38023.699023651694
mean_of_areas 905.3261672298022
iqr 600.7310742330737
plus outliersbox_area 3215.8441263418645
box {'coordinates': [3058.7525329589844, 3587.0962524414062, 3115.2474670410156, 3644.0189514160156], 'source': 'slices/2025_08_05_PhotoRieboR4_g201b201078_f003_259/2025_08_05_PhotoRieboR4_g201b201078_f003_259_slice_007_006_2766_3227.png'}
---


In [139]:
without_duplicates_and_outliers

39